In [1]:
from osgeo import gdal
from osgeo import ogr
import os
import numpy as np

In [5]:
def iterate_rasters(input_folder):
    """Provides two lists of files that go in pairs"""
    # Get a list of raster files in the input folder
    raster_files = [filename for filename in os.listdir(input_folder) if filename.endswith((".tif", ".tiff"))]

    # Create two empty lists
    first_file_list = []
    second_file_list = []

    # Iterate through pairs of current and next files
    for i in range(len(raster_files) - 1):
        current_file = os.path.join(input_folder, raster_files[i])
        next_file = os.path.join(input_folder, raster_files[i + 1])
        # If the second file is empty, finish
        if not next_file:
            print("Finished getting both values")
            continue
        else:
            first_file_list.append(current_file)
            second_file_list.append(next_file)

    return first_file_list, second_file_list


def read_csv(csv_file):
    rule_values_list = []
    with open(csv_file, 'r') as file:
        # next(csv_reader)  # Skip the header row
        for line in file:
            # Remove leading and trailing whitespaces, split the line into two values based on spaces, and convert them to floats
            rule_value_1, rule_value_2 = map(float, line.split(","))
            rule_values_list.append((rule_value_1, rule_value_2))
    return rule_values_list

def compare_rasters(first_file_list, second_file_list, output, rule_table_path):
    for raster1, raster2 in zip(first_file_list, second_file_list):
        # Open raster files
        ds1 = gdal.Open(raster1)
        ds2 = gdal.Open(raster2)

        if not ds1 or not ds2:
            print("Error: Unable to open raster files.")
        
        # Check if both rasters have the same height and width
        if ds1.RasterXSize != ds2.RasterXSize or ds1.RasterYSize != ds2.RasterYSize:
            print(f"Error: Rasters {raster1} and {raster2} have different dimensions.")
            continue
        
        # Get the first raster information
        width = ds1.RasterXSize
        height = ds1.RasterYSize
        geotransform = ds1.GetGeoTransform()
        projection = ds1.GetProjection()
        
        # Read rule table and create a list of pairs with the info
        rule_values_list = read_csv(rule_table_path)

        # Create output raster
        driver = gdal.GetDriverByName("GTiff")
        output_loc = os.path.join(output, os.path.basename(raster1.split('.')[0]) + "_" + os.path.basename(raster2.split('.')[0]))
        output_ds = driver.Create(output_loc, width, height, 1, gdal.GDT_Float32)
        output_ds.SetGeoTransform(geotransform)
        output_ds.SetProjection(projection)
        print("Created raster file")

        # Loop through each pixel and compare values

        block_size = 1000 #256  # Adjust the block size as needed
        for y in range(0, height, block_size):
            for x in range(0, width, block_size):
                print("we are in: {}_{}".format(str(y), str(x)))
                block1 = ds1.GetRasterBand(1).ReadAsArray(x, y, block_size, block_size)
                block2 = ds2.GetRasterBand(1).ReadAsArray(x, y, block_size, block_size)
        
                # Check if blocks are None
                if block1 is None or block2 is None:
                    print("Error: Failed to read raster blocks")
                    # To write down something
                    output_ds.GetRasterBand(1).WriteArray(np.array([[255]]), x, y)
                    continue
        
                # Check if the pair of values matches any rule pair
                correct_value = 1
                error_value = 2
                
                for i in range(block_size):
                    for j in range(block_size):
                        value1 = block1[i, j]
                        value2 = block2[i, j]

                        if (value1, value2) in rule_values_list:
                            output_ds.GetRasterBand(1).WriteArray(np.array([[error_value]]), x, y)
                        else:
                            output_ds.GetRasterBand(1).WriteArray(np.array([[correct_value]]), x, y)


        print("finished with {}".format(ds1))
        # Close datasets
        ds1 = None
        ds2 = None
        output_ds = None
        return


In [3]:
# Folder with all the rasters
raster_path = r"Z:\z_resources\im-nca-senegal\complete_trial\vector_union\lc_maps"
rule_table_path = r"C:\Users\ruben.crespo\Downloads\illogical_transitions.csv"
output_path = r"C:\Users\ruben.crespo\Downloads"

first_file_list, second_file_list = iterate_rasters(raster_path)
# print(first_file_list[0])
# print(second_file_list[0])
rule_values_list = read_csv(rule_table_path)

In [ ]:
compare_rasters(first_file_list, second_file_list, output_path, rule_table_path)

In [15]:
"""Ni caso, aquí estoy testeando"""
for raster1, raster2 in zip(first_file_list, second_file_list):
    # Open raster files
    ds1 = gdal.Open(raster1)
    ds2 = gdal.Open(raster2)

    if not ds1 or not ds2:
        print("Error: Unable to open raster files.")
    
    # Check if both rasters have the same height and width
    if ds1.RasterXSize != ds2.RasterXSize or ds1.RasterYSize != ds2.RasterYSize:
        print(f"Error: Rasters {raster1} and {raster2} have different dimensions.")
        continue
    
    # Get the first raster information
    width = ds1.RasterXSize
    height = ds1.RasterYSize
    geotransform = ds1.GetGeoTransform()
    projection = ds1.GetProjection()
    
    # Read rule table and create a list of pairs with the info
    rule_values_list = read_csv(rule_table_path)

    # Create output raster
    driver = gdal.GetDriverByName("GTiff")
    output_loc = os.path.join(output_path, os.path.basename(raster1.split('.')[0]) + "_" + os.path.basename(raster2.split('.')[0]))
    output_ds = driver.Create(output_loc, width, height, 1, gdal.GDT_Float32)
    output_ds.SetGeoTransform(geotransform)
    output_ds.SetProjection(projection)
    print("Created raster file")

    # Loop through each pixel and compare values

    block_size = 1000 #256  # Adjust the block size as needed
    for y in range(0, height, block_size):
        for x in range(0, width, block_size):
            print("we are in: {}_{}".format(str(y), str(x)))
            block1 = ds1.GetRasterBand(1).ReadAsArray(x, y, block_size, block_size)
            block2 = ds2.GetRasterBand(1).ReadAsArray(x, y, block_size, block_size)
    
            # Check if blocks are None
            if block1 is None or block2 is None:
                print("Error: Failed to read raster blocks")
                # To write down something
                output_ds.GetRasterBand(1).WriteArray(np.array([[255]]), x, y)
                continue
    
            # Check if the pair of values matches any rule pair
            correct_value = 1
            error_value = 2
            
            for i in range(block_size):
                for j in range(block_size):
                    value1 = block1[i, j]
                    value2 = block2[i, j]

                    if (value1, value2) in rule_values_list:
                        output_ds.GetRasterBand(1).WriteArray(np.array([[error_value]]), x, y)
                    else:
                        output_ds.GetRasterBand(1).WriteArray(np.array([[correct_value]]), x, y)


    print("finished with {}".format(ds1))
    # Close datasets
    ds1 = None
    ds2 = None
    output_ds = None


Created raster file
we are in: 0_0
we are in: 0_1000
we are in: 0_2000
we are in: 0_3000
we are in: 0_4000
we are in: 0_5000
we are in: 0_6000
we are in: 0_7000
we are in: 0_8000
we are in: 0_9000
we are in: 0_10000
we are in: 0_11000
we are in: 0_12000
we are in: 0_13000
Error: Failed to read raster blocks
we are in: 1000_0
we are in: 1000_1000
we are in: 1000_2000
we are in: 1000_3000
we are in: 1000_4000
we are in: 1000_5000
we are in: 1000_6000
we are in: 1000_7000
we are in: 1000_8000
we are in: 1000_9000
we are in: 1000_10000
we are in: 1000_11000
we are in: 1000_12000
we are in: 1000_13000
Error: Failed to read raster blocks
we are in: 2000_0
we are in: 2000_1000
we are in: 2000_2000
we are in: 2000_3000
we are in: 2000_4000
we are in: 2000_5000


In [9]:
print(output_loc)

Z:\z_resources\im-nca-senegal\complete_trial\vector_union\lc_maps\senegal_landcover_anat_32628_50m_2010_v2_Z:\z_resources\im-nca-senegal\complete_trial\vector_union\lc_maps\senegal_landcover_anat_32628_50m_2015_v2.tif
